# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [7]:
import json
with open ('/data/negotiated_rates.json') as f:
    data = json.load(f)
### here i am importing the JSON file so it can be used in the assignment###


In [8]:
data
### this is where I am calling the file###

{'reporting_entity_name': 'United HealthCare Services, Inc.',
 'reporting_entity_type': 'Third-Party Administrator',
 'last_updated_on': '2022-10-01',
 'version': '1.0.0',
 'out_of_network': [{'name': 'E-STIM 1/> AREAS OTH THAN WND CARE PART TX PLAN',
   'billing_code_type': 'HCPCS',
   'billing_code_type_version': '2022',
   'billing_code': 'G0283',
   'description': 'Electrical stimulation (unattended), to one or more areas for indication(s) other than wound care, as part of a therapy plan of care',
   'allowed_amounts': [{'tin': {'type': 'npi', 'value': '1184090458.0'},
     'service_code': '11',
     'billing_class': 'professional',
     'payments': {'allowed_amount': 8.78,
      'providers': [{'billed_charge': 10.0, 'npi': [1184090458.0]}]}}]},
  {'name': 'MANUAL THERAPY TQS 1/> REGIONS EACH 15 MINUTES',
   'billing_code_type': 'CPT',
   'billing_code_type_version': '2022',
   'billing_code': '97140',
   'description': 'Manual therapy techniques (eg, mobilization/ manipulation, ma

In [11]:
def average_rate(file):
    
    allowed_amounts = []
    for item in data['out_of_network']:
        item = item['allowed_amounts']
        allowed_amounts.append(item)      
    payments = []
    for item in allowed_amounts:
        for payment in item:
            payment = payment['payments']['allowed_amount']
            payments.append(payment)
#### the allowed amounts was in the JSON and I had to go through several layers to pull it out inclduing an area that was also called allowed amounts but the allowed amount was wihtin the payment area and labeled as allowed amount
    average = sum(payments)/len(payments)
    average = round(average,2)
### it was then requested to provide an average rate which i used an average function to create and also was asked to round the integer###
    return average

In [12]:
average_rate('/data/negotiated_rates.json') 
## i asserted my code and got the same answer as you##

38.67

In [3]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [31]:
import json

def get_rate(file, billing_code, service_code):
    
     with open (file) as f:
        data = json.load(f)

        allowed_amounts = []
        for item in data['out_of_network']:
                if item['billing_code'] == billing_code:
                    item = item['allowed_amounts']
                    allowed_amounts.append(item)

        payments = []
        for i, item in enumerate(allowed_amounts):
                 for payment in item:
                    if payment['service_code'] != service_code:
                        continue
                    payment = payment['payments']['allowed_amount']
                    payments.append(payment)
## i added in this average section for the payments so i was able to get the required rates##
        average = sum(payments)/len(payments)
        average = round(average,2)
        return average

## i used code from the previous problem to build onto this one so I could get the rate based on billing code and service code
## i also rounded the answer to 2 decimal places as stated to do so in the directions###

    

In [38]:
get_rate('/data/negotiated_rates.json','G0283','11')
### here i am asserting the get rate to ensure that my code is running##

8.78

In [36]:
get_rate('/data/negotiated_rates.json','97140','11')

20.03

In [37]:
get_rate('/data/negotiated_rates.json','97110','12')

26.62

In [ ]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [39]:
from datetime import datetime

def get_charge_rate(age, date):
    date = datetime.strptime(date, '%Y-%m-%d')
    if age >= 65:
        return 0.5
    elif date.weekday() == 0:
        return 0.75
    return 1
### i added this to my line of code because only certain prices should be returned based on age and day of the week so an if and elif statement was needed to ensure this##
def get_adjusted_rate(filename, billing_code, service_code, pat_age, visit_date):
    charge = get_charge_rate(pat_age, visit_date)
    import json
    with open (filename) as f:
        data = json.load(f)
## this is where I define what get adjusted rate is meaning so the rest of my code will run##
    allowed_amounts = []
    for item in data['out_of_network']:
        if item['billing_code'] == billing_code:
            item = item['allowed_amounts']
            allowed_amounts.append(item)

    payments = []
    for i, item in enumerate(allowed_amounts):
         for payment in item:
            if payment['service_code'] != service_code:
                continue
            payment = payment['payments']['allowed_amount']
            payments.append(payment)
    average = sum(payments)/len(payments)
    average*= charge
    average = round(average,2)
    return average
### everything else is code that was created in the previous 2 steps##

In [41]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07')
### I am asserting all of the problems to make sure that my code runs smoothly##

8.78

In [42]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03')

6.58

In [43]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03')

4.39

In [44]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08')

4.39

In [40]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [73]:

from datetime import datetime
CSV_FILE='/data/visits.csv'
   

In [72]:
help(csv.reader)

AttributeError: '_io.TextIOWrapper' object has no attribute 'reader'

In [67]:
with open(CSV_FILE) as csv:
    counter = 0
    max_lines = 300
    for line in csv:    
        values = line.strip().split(",")
        print(values)
        counter += 1
        if counter > max_lines:
            break
### this is where I am calling the CSV file to be used in my code below###

['SYSTEM', 'HOSPITAL', 'PATIENT_ID', 'PATIENT_AGE', 'BILLING_CODE', 'SERVICE_CODE', 'VISIT_DATE']
['Mercy', 'Missouri Baptist', 'E74109', '39', '97140', '12', '2021-05-01']
['BJC', 'SSM DePaul', 'E29207', '40', '97140', '11', '2021-03-25']
['SSM', 'SLU Hospital', 'E37839', '33', 'G0283', '11', '2021-07-25']
['SSM', 'Mercy St. Louis', 'E45853', '65', 'G0283', '12', '2021-07-27']
['SSM', 'SLU Hospital', 'E14769', '48', '97140', '12', '2021-06-16']
['BJC', 'Barnes Jewish', 'E60975', '52', '97140', '11', '2021-11-23']
['BJC', 'Barnes Jewish', 'E65110', '48', 'G0283', '11', '2021-06-02']
['BJC', 'Barnes Jewish', 'E25293', '52', '97140', '11', '2021-10-22']
['SSM', 'Barnes Jewish', 'E45548', '34', 'G0283', '11', '2021-12-31']
['BJC', 'Mercy Springfield', 'E13689', '49', 'G0283', '11', '2021-06-10']
['SSM', 'SLU Hospital', 'E64993', '59', 'G0283', '11', '2021-10-31']
['BJC', 'SSM DePaul', 'E76078', '60', '97140', '11', '2021-04-14']
['BJC', 'SSM DePaul', 'E52770', '59', 'G0283', '12', '2021-0

AttributeError: '_io.TextIOWrapper' object has no attribute 'reader'

In [68]:
def summarize_reimbursement(visits_data,negotiated_rates):
    by_month = {}
    by_hospital = {}
    
 ## here i defined sumarize reimbursment and then began the code to get the calcuated by month and hospital###   
    with open (visits_data,'r') as f:
        visits_data = list(csv_reader(f))
        
        allowed_amount = get_adjusted_rate(negotiated_rates, billing_code, service_code, patient_age, visit_date)
 ### i needed to define the allowed amount which was talked about in the earlier steps, to ensure that all of the right data was being pulled###       
        for individual_visit in pateintvisit_data[1:]:
            billing_code = individual_visit[4]
            service_code = individual_visit[5]
            patient_age = int(individual_visitt[3])
            visit_date = individual_visit[6]
            hospital = individual_visit[1]
 #### each entity like billing_code had a specific number that went with it to pull the right data so i have the defined above in order of how they are being pulled for allowed amounts###          
            if allowed_amount:
                month_data = visit_date[0:7]
                
                if month_data in by_month:
                    by_month[month_data] += allowed_amount
                else:
                    by_month[month_data] = allowed_amount
 ### i then wrote code to get just the year and month which took some time to figure out but if i used the month and year data which only went to 7 then it ended up working for me and left our the specific day###                   
            for month_data, value in by_month.items():
                by_month[month_data] = round (value,2)
            
                if hospital_data in by_hospital:
                    by_hospital[hospital_data] += allowed_amount
                else:
                    by_hospital[hospital_data] = allowed_amount
        ### this is where I broke the data down by hospital and was able to get the           
            for hospital_data, value in by_hospital.items():
                by_hospital[hospital_data] = round (value,2)
            return by_month,by_hospital

In [69]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

NameError: name 'csv_reader' is not defined

In [74]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

NameError: name 'csv_reader' is not defined

In [ ]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)